In [ ]:
import csv
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Fields list to store the names of the column field
fields = []
# Rows list to store the values
rows = []

# Path to the CSV file
mempool_path = 'gdrive/My Drive/Colab Notebooks/Summer of Bitcoin/mempool.csv'

# Reading the CSV file
with open(mempool_path, 'r') as csv_file:
  csv_reader = csv.reader(csv_file)
  fields = next(csv_reader)

  for row in csv_reader:
    rows.append(row)

In [ ]:
# Appending two new columns in the csv file
# Column val to store the ratio of weight/price, So that miner can input the child_idx in the form which has maxiumum fees for minumum weight
# A visited column to store if the child_idx id has been visited or not
for i in range(len(rows)):
  fee = int(rows[i][1])
  wgt = int(rows[i][2])
  val = fee / wgt
  rows[i].append(val)
  rows[i].append("False")

In [ ]:
# Sorting the data corresponding to Column val and reverse it, So that max fees transaction with min weigth appears on top.
new_data = sorted(rows, key = lambda x:x[4])
data = new_data[::-1]
for line in data:
  print(line)

In [ ]:
# Total_wgt in the tx_ids list should not be greater than the max_wgt
global total_wgt
total_wgt = 0
global max_wgt
max_wgt = 4000000
# tx_ids is a list that will store all the child_id that are the transaction ids used by minor to maximise the fees
global tx_ids
tx_ids = []

In [ ]:
# Function to calculate the Index value
def index_p_id(p_id):
  p_id = str(p_id)
  index_val = 0
  for i in range(len(data)):
    if (p_id == data[i][0]):
      index_val = i
      return index_val 

In [ ]:
# Using Recursion to store all the dependent parent transactions ids
def dependent_parent (p_id, que_dep_id):

  # Converting the p_id as child_id, so that transaction can occur before it
  # Calculating child_id_index for the following p_id
  p_id_index = index_p_id(p_id)
  if (p_id_index == 0):
    # Transaction cannot occur
    return que_dep_id
  else:
    new_child_id = data[p_id_index][0]
    parent_new_child_id = data[p_id_index][3]
    que_dep_id.append(new_child_id)

    # It is no further dependent on any parent transaction id
    if (parent_new_child_id == ""):
      return que_dep_id
    else:
      dependent_parent(new_child_id[value][3], que_dep_id)

In [ ]:
# Code to check if the Transaction is in list or not
def check_in_lst(id):
  for i in range(len(data)):
    if (id == data[i][0]):
      return True
  else:
    return False

In [ ]:
# Running a loop to iterate all the transation ids, If all the conditions are applied.
# We are following a 0 - based indexing approach

# Initialising i = 0 to i = len(data) ---> Total no of transaction ids
i = 0
while (i < len(data)):
  child_id = data[i][0]
  fee_id = data[i][1]
  wgt_id = int(data[i][2])
  parent_id = data[i][3]
  visited_id = data[i][5]

  # Case 1 --> The child_id is not dependent on Parent and we have not visited the transaction
  if (parent_id == "" and visited_id == "False"):
    # Transaction moves into tx_ids
    tx_ids.append(child_id)
    visited_id = "True"
    total_wgt += int(wgt_id)
  # Case 2 --> The child_is dependent on parent. So for child_id to move into tx_ids first parent_id should move to tx_ids
  elif (parent_id != "" and visited_id == "False"):
    parent_ids = list(map(str,parent_id.split(";")))
    for p_id in parent_ids:
      # p_id transaction already occured and is in tx_ids
      if (p_id in tx_ids):
        pass
      else:
        # p_id transaction has not occured till yet
        # queue to store the dependent parent transaction ids
        que_dep_pid = []
        # list contains all the dependent parent transaction ids
        lst = dependent_parent(p_id, que_dep_pid)
        # Append the lst in tx_id in the reverse order
        for i in range(len(lst)-1,-1,-1):
          t_id = lst[i]
          # This transaction is now visited, So visited_id = "True"
          # Checking if this transaction is in list or not
          in_lst = check_in_lst(t_id)
          if (in_lst == "True"):
            ind = data[0].index(t_id)
            data[ind][5] = "True"
            # Adding this transaction in tx_ids
            tx_ids.append(t_id)
            total_wgt += int(data[ind][2])
          else:
            pass
    # Case 3 --> The child id is dependent on parent. But the transaction has already occured before.
    else: 
      pass
  
  # Keeping in mind the weight factor
  if (total_wgt > max_wgt):
    break
  else:
    pass 
  i += 1      

#### The tx_ids list store all the transactions done by the Miner
#### The Block.txt file store the list of all the transactions

In [ ]:
Block_file_path = 'gdrive/My Drive/Colab Notebooks/Summer of Bitcoin/Block.txt'

In [ ]:
textfile = open(Block_file_path, "w")
for element in tx_ids:
    textfile.write(element + "\n")
textfile.close()